In [1]:
import random
import math
import logging
import time
import tensorflow as tf
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

data = pd.read_csv('../projet_recherche/sampled_data.csv')
data.head()

data_train = data[data['building_id'] != 8]
data_test = data[data['building_id'] == 8]

data_train.shape, data_test.shape


target_column = 'production'

x_train = data_train.drop(target_column, axis=1)
y_train = data_train[target_column].values.reshape(-1, 1)

x_test = data_test.drop(target_column, axis=1)
y_test = data_test[target_column].values.reshape(-1, 1)


x_scaler = MinMaxScaler(feature_range=(0, 1))
x_scaler.fit(x_train)

x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)


def get_windows(x, y, window_size):
    x_windows, y_windows = [], []

    for i in range(len(x) - window_size):
        x_window = x[i:i+window_size]
        y_window = y[i:i+window_size]

        x_window = np.hstack((x_window, y_window))

        x_windows.append(x_window)
        y_windows.append(y[i+window_size])

    return np.array(x_windows), np.array(y_windows)


x_train_windows, y_train_windows = get_windows(x_train_scaled, y_train, 10)
x_test_windows, y_test_windows = get_windows(x_test_scaled, y_test, 10)


tf.random.set_seed(42)


def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    print(f"MSE: {mse}")
    return mse


# Define the parameter ranges
LSTM1_units = list(range(16, 256, 8))
LSTM1_activation = ['tanh', 'relu', 'sigmoid', 'hard_sigmoid', 'linear']
DROPOUT1_rate = list(np.arange(0, 1, 0.01))
LSTM2_units = list(range(16, 256, 8))
LSTM2_activation = ['tanh', 'relu', 'sigmoid', 'hard_sigmoid', 'linear']
DROPOUT2_rate = list(np.arange(0, 1, 0.01))
DENSE1_units = list(range(16, 256, 8))
DENSE1_activation = ['tanh', 'relu', 'sigmoid', 'hard_sigmoid', 'linear']
DENSE2_units = [1]
DENSE2_activation = ['linear']
OPTIMIZER_learning_rate = list(np.arange(0, 0.001, 0.0001))
EPOCHS = list(range(200, 400, 10))
BATCH_SIZE = list(range(32, 128, 8))


def objective_function(params):
    print(f"params {params}")
    # Unpack the parameters
    lstm1_units, lstm1_activation, dropout1_rate, lstm2_units, lstm2_activation, dropout2_rate, dense1_units, dense1_activation, dense2_units, dense2_activation, optimizer_learning_rate, epochs, batch_size = params

    model = Sequential([
        LSTM(lstm1_units, activation=lstm1_activation, input_shape=(
            x_train_windows.shape[1:]), return_sequences=True),
        Dropout(dropout1_rate),

        LSTM(lstm2_units, activation=lstm2_activation, return_sequences=False),
        Dropout(dropout2_rate),

        Dense(dense1_units, activation=dense1_activation),
        Dense(dense2_units, activation=dense2_activation)
    ])

    optimizer = Adam(learning_rate=optimizer_learning_rate)
    model.compile(optimizer=optimizer, loss='mean_absolute_error')

    try:

        start_time = time.time()
        history = model.fit(x=x_train_windows,
                            y=y_train_windows,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_split=0.2,
                            shuffle=False,
                            verbose=0)
        training_duration = time.time() - start_time
    finally:
        print("finito pipo")
        # tracker.stop()

    y_pred_train = model.predict(x_train_windows)
    y_pred_test = model.predict(x_test_windows)
    performance_metric = evaluate_model(y_test_windows, y_pred_test)

    return performance_metric


def simulated_annealing(initial_temp, final_temp, alpha, max_iterations):
    # Initialize the default solution 
    current_solution = [
        128, "tanh", 0.2,
        64, "tanh", 0.2,
        64, "relu", 1, "linear",
        0.001, 300, 128
    ]
    current_mse = objective_function(current_solution)
    best_solution = current_solution
    best_mse = current_mse
    temp = initial_temp

    for iteration in range(max_iterations):
        if temp < final_temp:
            break

        # Generate a neighboring solution
        neighbor = list(current_solution)
        param_index = random.randint(0, len(current_solution) - 1)
        if param_index == 0:
            neighbor[param_index] = random.choice(LSTM1_units)
        elif param_index == 1:
            neighbor[param_index] = random.choice(LSTM1_activation)
        elif param_index == 2:
            neighbor[param_index] = random.choice(DROPOUT1_rate)
        elif param_index == 3:
            neighbor[param_index] = random.choice(LSTM2_units)
        elif param_index == 4:
            neighbor[param_index] = random.choice(LSTM2_activation)
        elif param_index == 5:
            neighbor[param_index] = random.choice(DROPOUT2_rate)
        elif param_index == 6:
            neighbor[param_index] = random.choice(DENSE1_units)
        elif param_index == 7:
            neighbor[param_index] = random.choice(DENSE1_activation)
        elif param_index == 8:
            neighbor[param_index] = random.choice(DENSE2_units)
        elif param_index == 9:
            neighbor[param_index] = random.choice(DENSE2_activation)
        elif param_index == 10:
            neighbor[param_index] = random.choice(OPTIMIZER_learning_rate)
        elif param_index == 11:
            neighbor[param_index] = random.choice(EPOCHS)
        elif param_index == 12:
            neighbor[param_index] = random.choice(BATCH_SIZE)

        neighbor_mse = objective_function(neighbor)

        # Acceptance criterion

        mse_diff = neighbor_mse - current_mse
        print(f"current_solution {current_solution}")
        print(f"neighbor {neighbor}")
        print(f"mse diff: {mse_diff}")
        if mse_diff < 0 or random.random() < math.exp(-mse_diff / temp):
            print("accept new solution", neighbor)
            current_solution = neighbor
            current_mse = neighbor_mse

            if neighbor_mse < best_mse:
                print("better solution")
                best_solution = neighbor
                best_mse = neighbor_mse

        # Update temperature
        temp = temp * alpha
        print(f"temp {temp}, mse {current_mse}, best_mse {best_mse}")

    return best_solution, best_mse


# Run the simulated annealing algorithm
initial_temp = 20  # Initial temperature
final_temp = 0.2  # Final temperature
alpha = 0.9  # Temperature decay rate
max_iterations = 10000  # Maximum number of iterations

best_params, best_mse = simulated_annealing(
    initial_temp, final_temp, alpha, max_iterations)
print("Best parameters:", best_params)
print("Best mse:", best_mse)

2024-05-15 15:47:58.195397: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 15:48:03.619732: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


params [128, 'tanh', 0.2, 64, 'tanh', 0.2, 64, 'relu', 1, 'linear', 0.001, 300, 128]


2024-05-15 15:48:10.459514: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-15 15:48:10.511297: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-15 15:48:10.511596: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

finito pipo
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE: 6.932248870926603
params [128, 'tanh', 0.2, 64, 'tanh', 0.2, 64, 'relu', 1, 'linear', 0.0004, 300, 128]


/home/quentin/Desktop/Optimization/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


finito pipo
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
MSE: 14.245784465258888
current_solution [128, 'tanh', 0.2, 64, 'tanh', 0.2, 64, 'relu', 1, 'linear', 0.001, 300, 128]
neighbor [128, 'tanh', 0.2, 64, 'tanh', 0.2, 64, 'relu', 1, 'linear', 0.0004, 300, 128]
mse diff: 7.313535594332285
accept new solution [128, 'tanh', 0.2, 64, 'tanh', 0.2, 64, 'relu', 1, 'linear', 0.0004, 300, 128]
temp 18.0, mse 14.245784465258888, best_mse 6.932248870926603
params [128, 'tanh', 0.2, 64, 'tanh', 0.2, 64, 'relu', 1, 'linear', 0.0004, 300, 128]


/home/quentin/Desktop/Optimization/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


finito pipo


KeyboardInterrupt: 